In [1]:

from dotenv import load_dotenv, dotenv_values
from metagame import TradingClient
import os

env = dotenv_values()

client = TradingClient(
    api_url=env["API_URL"],
    jwt=env["JWT"],
    act_as=int(env["ACT_AS"]),
)

In [2]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    RED = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

print(bcolors.OKGREEN + "Warning: No active frommets remain. Continue?" + bcolors.ENDC)


In [3]:
import pandas as pd

def get_trades(state, market_id:int):
    if not state.markets[market_id].hasFullTradeHistory:
        client.get_full_trade_history(market_id)
        state = client.state()
    return state.markets[market_id].trades

def get_market_trades(market_name: str):
    state = client.state()
    market_id = state.market_name_to_id.get(market_name)
    if market_id is None:
        return pd.DataFrame()
    
    trades = get_trades(state, market_id)
    
    if not trades:
        return pd.DataFrame()
    df = pd.DataFrame(trades).set_index("id")
    df["buyer"] = df["buyer_id"].map(lambda x: state.accounts[x].name)
    df["seller"] = df["seller_id"].map(lambda x: state.accounts[x].name)
    df["timestamp"] = df["transaction_id"].map(lambda x: state.transactions[x])
    return df.drop(columns=["market_id", "buyer_id", "seller_id", "transaction_id"])

In [4]:
def get_market_positions(market_name: str):
    df = get_market_trades(market_name)
    if df.empty:
        return pd.DataFrame()
    return pd.concat([
        df.groupby("buyer")["size"].sum(),
        -df.groupby("seller")["size"].sum()
    ]).groupby(level=0).sum()

def get_multiple_market_positions(market_names: list[str]):
    return pd.DataFrame({market: get_market_positions(market) for market in market_names}).fillna(0)

In [5]:
import ipywidgets as widgets

def get_dice(refresh):
    markets = ["min_dice_c", "max_dice_c", "avg_dice_c"]
    return get_multiple_market_positions(markets)

# widgets.interact(get_dice, refresh=widgets.Checkbox(description="Refresh Data"))

In [16]:
side_dict= {1:"bid", 2:"offer"}

def get_prices(market):
    # highest bid
    try:
        bid = max((order for order in market.orders if order.side==1), key=lambda x: -x.price)
        bid_price = bid.price
        bid_size = bid.size
    except IndexError:
        bid_price = -float("inf")
        bid_size = 0
    # lowest offer
    try:
        offer = min((order for order in market.orders if order.side==2), key=lambda x: x.price)
        offer_price = offer.price
        offer_size = offer.size
    except IndexError:
        offer_price = float("inf")
        offer_size = 0
    return {"bid": bid_price, "offer": offer_price, "bid_size": bid_size, "offer_size": offer_size}


def get_market_prices(client, /, market_name: str | None = None, market_id: int | None = None):
    state = client.state()
    if market_id is None:
        market_id = state.market_name_to_id.get(market_name)
    market = state.markets[market_id]
    return get_prices(market)

In [7]:
from pydantic import BaseModel, ConfigDict

s MarketPrices(BaseModel):
    id: int
    name: str
    bid: float
    offer: float
    bid_size: float = 0
    offer_size: float = 0
    model_config = ConfigDict(arbitrary_types_allowed=True)

    @classmethod
    def from_client(cls, client, /, name: str | None = None, id: int | None = None):
        state = client.state()
        if id is None:
            id = state.market_name_to_id.get(name)
        prices = get_market_prices(client, market_id=id)

        return cls(#client = client, 
                         id = id,
                         name=name,
                         bid = prices["bid"], 
                         offer = prices["offer"],
                         bid_size = prices["bid_size"],
                         offer_size = prices["offer_size"]
                         )
    
    def __add__(self, other):
        return MarketPrices(id=-1, name=self.name + ' + ' + other.name, 
                            bid=self.bid + other.bid, offer=self.offer + other.offer)
    
    def __mul__(self, other:float|int):
        
        return MarketPrices(id=-1, name=self.name + ' * ' + str(other), 
                            bid=self.bid * other, offer=self.offer * other)
    def __sub__(self, other):
        return MarketPrices(id=-1, name=self.name + ' - ' + other.name, 
                            bid=self.bid - other.bid, offer=self.offer - other.offer)


In [8]:
market_name_a = "test_mix_max"
prices_a = MarketPrices.from_client(client, market_name_a)
# prices_a * 2

state = client.state()
market_name_b = "Will any team go bankrupt via a rogue bot?"
prices_b = MarketPrices.from_client(client, market_name_b)
# prices_b

state = client.state()
market_name_c = "Will any team go bankrupt via a rogue bot?"
prices_c = MarketPrices.from_client(client, market_name_c)


In [14]:
etf_2a_b_c = prices_a * 2 + prices_b + prices_c
etf_2a_b_c

MarketPrices(id=-1, name='test_mix_max * 2 + Will any team go bankrupt via a rogue bot? + Will any team go bankrupt via a rogue bot?', bid=0.0, offer=220.0)

In [13]:
raw_2a_b_c = prices_a * 1.4 + prices_b + prices_c
raw_2a_b_c

MarketPrices(id=-1, name='test_mix_max * 1.4 + Will any team go bankrupt via a rogue bot? + Will any team go bankrupt via a rogue bot?', bid=0.0, offer=160.0)

In [15]:
print(#"buy ETF(2a+b) and sell raw (2a+b)",
      bcolors.OKGREEN + "buy ETF(2a + b + c) " + bcolors.ENDC,
      bcolors.RED + "sell raw (2a + b + c) " + bcolors.ENDC,
      etf_2a_b_c.bid - raw_2a_b_c.offer
      )

print(#"buy raw (2a+b) and sell ETF(2a+b)",
      bcolors.OKGREEN + "raw ETF(2a + b) " + bcolors.ENDC,
      bcolors.RED + "sell ETF (2a + b) " + bcolors.ENDC,
      raw_2a_b_c.bid - etf_2a_b_c.offer
      )

buy ETF(2a + b + c)  sell raw (2a + b + c)  -160.0
raw ETF(2a + b)  sell ETF (2a + b)  -220.0


In [28]:
etf_2a_b.offer - raw_2a_b.bid

210.0

In [8]:
factor = 5
fee = 1

market_name_a = "talya_clips"

market_id_a = state.market_name_to_id.get(market_name_a)
market_a = state.markets[market_id_a]
prices_a = get_prices(market_a)

market_name_b = "age of youngest attendee"
market_id_b = state.market_name_to_id.get(market_name_b)
market_b = state.markets[market_id_b]


prices_b = get_prices(market_b)

# prices_a = {"offer": 1, "bid": 2} # avg
# prices_b = {"offer": 4.5, "bid": 6} # sum

#                    avg                 sum
a_to_b = - factor * prices_a["offer"] + prices_b["bid"] - fee

#                  sum                 avg
b_to_a = - prices_b["offer"] + factor * prices_a["bid"] - fee

# print(f"{market_name_a} -> {market_name_b}: {a_to_b}")
# print(f"{market_name_b} -> {market_name_a}: {b_to_a}")
print(f"buy sum / sell avg: {a_to_b}")
print(f"buy avg / sell sum: {b_to_a}")

buy sum / sell avg: -377.5
buy avg / sell sum: 124.6
